### Since the file exceeds GitHub's maximum size limit, it has been added to the .gitignore. To run the notebook and the functions, please add the data file to the src folder of the project.

In [ ]:
# Standard library imports
import gc
import heapq
import json
import re
from collections import defaultdict, Counter
from datetime import datetime
from typing import List, Tuple

# Third-party imports
import emoji
import memray
import orjson
import pandas as pd
import snakeviz
import ujson

In [ ]:
file_path = "farmers-protest-tweets-2021-2-4.json"

### First, I tried to load the file in a pandas to evaluate if the file was correct.

In [ ]:
df = pd.read_json(file_path)

### As the load fails I checked the file to find what happened. I used an online tool to prove some random objects and I saw that each object was correct. So I assumed that the problem was that the file contained a lot of objects but them were not in a list. In the next cell I read the file and fix it adding each element in a list and convert to a json. I save the file again in a new file.

In [ ]:
# List to save the elements
json_list = []

# Read the file and add each JSON to the list
with open(file_path, 'r') as file:
    for line in file:
        try:
            # Load each line as JSON object and append to the list
            object_ = json.loads(line)
            json_list.append(object_)
        except json.JSONDecodeError as e:
            print(f'Error decoding the line: {line.strip()} - {e}')

In [ ]:
# Convert the list to JSON
valid_json = json.dumps(json_list, indent=4)

In [ ]:
with open('fixed_data.json', 'w') as end_file:
    end_file.write(valid_json)

In [ ]:
file_path_test = "fixed_data.json"

In [ ]:
df = pd.read_json(file_path_test)

In [ ]:
df.head()

In [ ]:
print(min(df["date"]), max(df["date"]))

### After processing the file like I explain before. I tried to load in a pandas again and this time it was correct. So I was able to visualize the file structure, fields, types and validate the date range of the file as well as the quantity of rows.

### After understanding the problem with the file I solved the first exercise knowing the problems that the file had and knowing that I had to correct it during the execution of the function.

### I defined a function to process the file in a way to optimize the execution time. I proved three packages to use the one that had better performance.

### I proved with json, ujson and orjson

### Finally, after checked the execution time of all of them. I decided to use orjson

In [ ]:
def process_json(file_path):
    tweets_per_date = defaultdict(lambda: defaultdict(int))
    # Open the file in read mode
    with open(file_path, 'r') as f:
        # Read the file line by line
        for line in f:
            # Fix and convert each line in a JSON object
            line = line.strip()
            if not line:
                # Ignore empty lines
                continue

            # Validate the object to process correctly
            if line.startswith('{') and line.endswith('}'):
                try:
                    tweet = orjson.loads(line)
                except orjson.JSONDecodeError as e:
                    print(f'Error decoding the line: {line.strip()} - {e}')

                # Obtain date and username of each tweet
                tweet_date = datetime.strptime(tweet['date'], '%Y-%m-%dT%H:%M:%S+00:00').date()
                username = tweet['user']['username']

                # Add to the count of that date and user
                tweets_per_date[tweet_date][username] += 1

    return tweets_per_date

## Exercise 1

### Now I create the function to return the top 10 of dates with more tweets and the username with more publications. First, I implemented a loop to extract username and date of each tweet and append to a defaultdict. Here I add a new parameter to the function called "version" to proved two versions that I implemented. The first one has a loop over the items of the defaultdict and for each date calculate the total tweets and the user with more tweets, then make a validation with the maximum found. The second one sorted the dates by the total tweets and then just for the top 10 calculate the user with more tweets

In [ ]:
def q1_time(file_path: str, version: int) -> List[Tuple[datetime.date, str]]:
    tweets_per_date = process_json(file_path)

    if version == 1:
        top_dates = []
        for date, users in tweets_per_date.items():
            # Calculate total tweets and find user with more tweets
            total_tweets = 0
            top_user = None
            max_tweets = 0

            for user, count in users.items():
                total_tweets += count
                if count > max_tweets:
                    max_tweets = count
                    top_user = user

            # Add date, total tweets and user to the list
            top_dates.append((date, total_tweets, top_user))

        # Sort list by total tweets and save just top 10
        top_dates_sorted = sorted(top_dates, key=lambda x: x[1], reverse=True)[:10]

        # Return date and user in a list of tuples
        return [(date, top_user) for date, _, top_user in top_dates_sorted]

    else:
        # List to save dates and total tweets per date
        date_tweet_sums = []
        # Dict to save users with more tweets per date
        top_users_by_date = {}

        # Calculate total tweets and find user with more tweets
        for date, users in tweets_per_date.items():
            total_tweets = sum(users.values())
            top_user = max(users.items(), key=lambda item: item[1])[0]
            date_tweet_sums.append((date, total_tweets))
            top_users_by_date[date] = top_user

        # Sort list by total tweets and save just top 10
        top_10_dates = sorted(date_tweet_sums, key=lambda x: x[1], reverse=True)[:10]

        # Use the date list and users dict to return the list
        return [(tweet_date, top_users_by_date[tweet_date]) for tweet_date, _ in top_10_dates]

### Now, for the function optimized by memory use I implemented a function that read the file line by line with a generator. The "tweet_generator" function process the file line by line and use yield to return each tweet one by one. With the use of yield avoid that the data are in memory and allow to procees when is necessary. In addition I used heap to implement a priority queue, where the element with the minimum or maximum value can be accessed efficiently, to obtain the k largest or smallest elements from a list efficiently

In [ ]:
def tweet_generator(file_path):
    # Open file in read mode
    with open(file_path, 'r') as f:
        # Read each line
        for line in f:
            # Delete line breaks and spaces
            line = line.strip()
            if not line:
                continue

            # Validate JSON format
            if line.startswith('{') and line.endswith('}'):
                # Convert to object
                try:
                    tweet = json.loads(line)
                except json.JSONDecodeError as e:
                    print(f'Error decoding the line: {line.strip()} - {e}')

                # Save the date and username
                tweet_date = datetime.strptime(tweet['date'], '%Y-%m-%dT%H:%M:%S+00:00').date()
                username = tweet['user']['username']

                # Use of yield to return data one by one
                yield tweet_date, username

def process_tweets(file_path):
    # Dict to count tweets by date and user
    tweets_per_date = defaultdict(lambda: defaultdict(int))

    # use generator to process each tweet
    for tweet_date, username in tweet_generator(file_path):
        # Add to dict
        tweets_per_date[tweet_date][username] += 1

    return tweets_per_date

def q1_memory(file_path):
    top_dates_heap = []
    tweets = process_tweets(file_path)

    # Process file in an incremental way and free memory whenever possible
    for date, user_data in tweets.items():
        # Total tweets by user
        total_tweets = sum(user_data.values())
        top_user = max(user_data, key=user_data.get)
        top_user_count = user_data[top_user]

        # Use heap
        heapq.heappush(top_dates_heap, (total_tweets, date, top_user, top_user_count))

        # Limit the heap to 10 elements
        if len(top_dates_heap) > 10:
            heapq.heappop(top_dates_heap)

        # Free memory manually
        del user_data
        gc.collect()

    # Convert the heap to sorted list
    top_dates_sorted = sorted(top_dates_heap, key=lambda x: x[0], reverse=True)

    return [(date, top_user) for _, date, top_user, _ in top_dates_sorted]

In [ ]:
q1_time(file_path=file_path, version=2)

In [ ]:
q1_memory(file_path=file_path)

### Use of snakeviz. SnakeViz is a browser based graphical viewer for the output of Python’s cProfile module. With that package we can see each function and their times

In [ ]:
%load_ext snakeviz

### Here I use snakeviz to see the time profile of each function. I run the time function twice, because I was proving the two versions

In [ ]:
%snakeviz q1_time(file_path=file_path, version=1)

In [ ]:
%snakeviz q1_time(file_path=file_path, version=2)

In [ ]:
%snakeviz q1_memory(file_path=file_path)

### Use of memray to see the memory used profile

In [ ]:
%load_ext memray

In [ ]:
%%memray_flamegraph
def q1_time(file_path: str, version: int) -> List[Tuple[datetime.date, str]]:
    tweets_per_date = process_json(file_path)

    if version == 1:
        top_dates = []
        for date, users in tweets_per_date.items():
            # Calculate total tweets and find user with more tweets
            total_tweets = 0
            top_user = None
            max_tweets = 0

            for user, count in users.items():
                total_tweets += count
                if count > max_tweets:
                    max_tweets = count
                    top_user = user

            # Add date, total tweets and user to the list
            top_dates.append((date, total_tweets, top_user))

        # Sort list by total tweets and save just top 10
        top_dates_sorted = sorted(top_dates, key=lambda x: x[1], reverse=True)[:10]

        # Return date and user in a list of tuples
        return [(date, top_user) for date, _, top_user in top_dates_sorted]

    else:
        # List to save dates and total tweets per date
        date_tweet_sums = []
        # Dict to save users with more tweets per date
        top_users_by_date = {}

        # Calculate total tweets and find user with more tweets
        for date, users in tweets_per_date.items():
            total_tweets = sum(users.values())
            top_user = max(users.items(), key=lambda item: item[1])[0]
            date_tweet_sums.append((date, total_tweets))
            top_users_by_date[date] = top_user

        # Sort list by total tweets and save just top 10
        top_10_dates = sorted(date_tweet_sums, key=lambda x: x[1], reverse=True)[:10]

        # Use the date list and users dict to return the list
        return [(tweet_date, top_users_by_date[tweet_date]) for tweet_date, _ in top_10_dates]

q1_time(file_path=file_path, version=2)

In [ ]:
%%memray_flamegraph
def tweet_generator(file_path):
    # Open file in read mode
    with open(file_path, 'r') as f:
        # Read each line
        for line in f:
            # Delete line breaks and spaces
            line = line.strip()
            if not line:
                continue

            # Validate JSON format
            if line.startswith('{') and line.endswith('}'):
                try:
                    # Convert to object
                    tweet = json.loads(line)
                except json.JSONDecodeError as e:
                    print(f'Error decoding the line: {line.strip()} - {e}')

                # Save the date and username
                tweet_date = datetime.strptime(tweet['date'], '%Y-%m-%dT%H:%M:%S+00:00').date()
                username = tweet['user']['username']

                # Use of yield to return data one by one
                yield tweet_date, username

def process_tweets(file_path):
    # Dict to count tweets by date and user
    tweets_per_date = defaultdict(lambda: defaultdict(int))

    # use generator to process each tweet
    for tweet_date, username in tweet_generator(file_path):
        # Add to dict
        tweets_per_date[tweet_date][username] += 1

    return tweets_per_date

def q1_memory(file_path):
    top_dates_heap = []
    tweets = process_tweets(file_path)

    # Process file in an incremental way and free memory whenever possible
    for date, user_data in tweets.items():
        # Total tweets by user
        total_tweets = sum(user_data.values())
        top_user = max(user_data, key=user_data.get)
        top_user_count = user_data[top_user]

        # Use heap
        heapq.heappush(top_dates_heap, (total_tweets, date, top_user, top_user_count))

        # Limit the heap to 10 elements
        if len(top_dates_heap) > 10:
            heapq.heappop(top_dates_heap)

        # Free memory manually
        del user_data
        gc.collect()

    # Convert the heap to sorted list
    top_dates_sorted = sorted(top_dates_heap, key=lambda x: x[0], reverse=True)

    return [(date, top_user) for _, date, top_user, _ in top_dates_sorted]

q1_memory(file_path=file_path)

### After use the packages, we can see that in terms of time the function q1_time is more efficient than q1_memory. However, in terms of memory we can see in the profiling that both functions has the same use of memory that is because in both cases I read the file line by line so just save in memory the last object that I processed. I try to load all the file in memory in q1_time but the orjson.loads function took a lot of time trying to convert the entire file to an object.

## Exercise 2

### For the second exercise I implemented a function to extract emojis from a text. Then I process the file line by line and extract emojis from the text and save in a Counter.

### For this exercise I assumed that I should to find the most used emojis regardless of whether they were in the main tweet or in a quoted tweet.

### I used the emoji package because even the use of regular expressions can be more efficient, that way has a complexity in maintain uploaded the list of expressions. The package emoji is uploaded with each new version of unicode

In [ ]:
def extract_emojis(text):
    """Extract emojis from text."""
    # Use emoji package to find emojis
    return [char for char in text if char in emoji.EMOJI_DATA]

def q2_time(file_path):
    emoji_counter = Counter()
    with open(file_path, 'r') as f:
            # Read the file line by line
            for line in f:
                # Fix and convert each line in a JSON object
                line = line.strip()
                if not line:
                    # Ignore empty lines
                    continue

                # Validate the object to process correctly
                if line.startswith('{') and line.endswith('}'):
                    try:
                        tweet = orjson.loads(line)
                    except orjson.JSONDecodeError as e:
                        print(f'Error decoding the line: {line.strip()} - {e}')

                # Extract emojis from 'content'
                content_emojis = extract_emojis(tweet.get('content', ''))
                emoji_counter.update(content_emojis)

                # Check if quotedTweet exists and extract emojis from their 'content'
                quoted_tweet = tweet.get('quotedTweet')
                if quoted_tweet:
                    quoted_content_emojis = extract_emojis(quoted_tweet.get('content', ''))
                    emoji_counter.update(quoted_content_emojis)

            # Obtain top 10 of emojis
            top_10 = emoji_counter.most_common(10)

            return top_10

In [ ]:
q2_time(file_path)

### For the memory function I used the same approach. I extract emojis with the emoji package from the content of the main tweet and the quoted tweet. And I used generator to process the file in incrementally way

In [ ]:
def extract_emojis(text):
    """Extract emojis from text."""
    return [char for char in text if char in emoji.EMOJI_DATA]

def tweet_generator(file_path):
    """Generator to process each tweet and their quotedTweet."""
    with open(file_path, 'r') as f:
            # Read the file line by line
            for line in f:
                # Fix and convert each line in a JSON object
                line = line.strip()
                if not line:
                    # Ignore empty lines
                    continue

                # Validate the object to process correctly
                if line.startswith('{') and line.endswith('}'):
                    try:
                        tweet = json.loads(line)
                    except json.JSONDecodeError as e:
                        print(f'Error decoding the line: {line.strip()} - {e}')

                # Process emojis from 'content'
                yield extract_emojis(tweet.get('content', ''))

                # Process emojis from 'content' in the 'quotedTweet' if it exists
                quoted_tweet = tweet.get('quotedTweet')
                if quoted_tweet:
                    yield extract_emojis(quoted_tweet.get('content', ''))

def q2_memory(file_path):
    emoji_counter = Counter()

    # Use generator to process tweets one by one
    for emoji_list in tweet_generator(file_path):
        for emoji_char in emoji_list:
            emoji_counter[emoji_char] += 1

    # Obtain top 10 of emojis
    return emoji_counter.most_common(10)

In [ ]:
q2_memory(file_path)

In [ ]:
%snakeviz q2_time(file_path)

In [ ]:
%snakeviz q2_memory(file_path)

In [ ]:
%%memray_flamegraph
def extract_emojis(text):
    """Extract emojis from text."""
    # Use emoji package to find emojis
    return [char for char in text if char in emoji.EMOJI_DATA]

def q2_time(file_path):
    emoji_counter = Counter()
    with open(file_path, 'r') as f:
            # Read the file line by line
            for line in f:
                # Fix and convert each line in a JSON object
                line = line.strip()
                if not line:
                    # Ignore empty lines
                    continue

                # Validate the object to process correctly
                if line.startswith('{') and line.endswith('}'):
                    try:
                        tweet = orjson.loads(line)
                    except orjson.JSONDecodeError as e:
                        print(f'Error decoding the line: {line.strip()} - {e}')

                # Extract emojis from 'content'
                content_emojis = extract_emojis(tweet.get('content', ''))
                emoji_counter.update(content_emojis)

                # Check if quotedTweet exists and extract emojis from their 'content'
                quoted_tweet = tweet.get('quotedTweet')
                if quoted_tweet:
                    quoted_content_emojis = extract_emojis(quoted_tweet.get('content', ''))
                    emoji_counter.update(quoted_content_emojis)

            # Obtain top 10 of emojis
            top_10 = emoji_counter.most_common(10)

            return top_10

q2_time(file_path)

In [ ]:
%%memray_flamegraph
def extract_emojis(text):
    """Extract emojis from text."""
    return [char for char in text if char in emoji.EMOJI_DATA]

def tweet_generator(file_path):
    """Generator to process each tweet and their quotedTweet."""
    with open(file_path, 'r') as f:
            # Read the file line by line
            for line in f:
                # Fix and convert each line in a JSON object
                line = line.strip()
                if not line:
                    # Ignore empty lines
                    continue

                # Validate the object to process correctly
                if line.startswith('{') and line.endswith('}'):
                    try:
                        tweet = json.loads(line)
                    except json.JSONDecodeError as e:
                        print(f'Error decoding the line: {line.strip()} - {e}')

                # Process emojis from 'content'
                yield extract_emojis(tweet.get('content', ''))

                # Process emojis from 'content' in the 'quotedTweet' if it exists
                quoted_tweet = tweet.get('quotedTweet')
                if quoted_tweet:
                    yield extract_emojis(quoted_tweet.get('content', ''))

def q2_memory(file_path):
    emoji_counter = Counter()

    # Use generator to process tweets one by one
    for emoji_list in tweet_generator(file_path):
        for emoji_char in emoji_list:
            emoji_counter[emoji_char] += 1  # Contar emojis uno por uno

    # Obtain top 10 of emojis
    return emoji_counter.most_common(10)

q2_memory(file_path)

### In the second exercise, after use snakeviz we can see that the function q2_time has a better performance in terms of execution time. For memory, in the profiling we can see that the function q2_memory has a better use of memory but the difference is 2 KiB. In large data sets there could be more difference

## Exercise 3

### For exercise 3 like in exercise 2 I assumed that I should to find the most influential users regardless of whether they were in the main tweet or in a quoted tweet.

In [ ]:
def q3_time(file_path):
    mention_counter = Counter()

    with open(file_path, 'r') as f:
            # Read the file line by line
            for line in f:
                # Fix and convert each line in a JSON object
                line = line.strip()
                if not line:
                    # Ignore empty lines
                    continue

                # Validate the object to process correctly
                if line.startswith('{') and line.endswith('}'):
                    try:
                        tweet = orjson.loads(line)
                    except orjson.JSONDecodeError as e:
                        print(f'Error decoding the line: {line.strip()} - {e}')

                # Process mentions in 'mentionedUsers' from main tweet
                mentioned_users = tweet.get('mentionedUsers', [])
                if mentioned_users is None:
                    mentioned_users = []
                mention_counter.update(user['username'] for user in mentioned_users)

                # Process mentions in 'mentionedUsers' from 'quotedTweet' if it exists
                quoted_tweet = tweet.get('quotedTweet')
                if quoted_tweet:
                    quoted_mentioned_users = quoted_tweet.get('mentionedUsers', [])
                    if quoted_mentioned_users is None:
                        quoted_mentioned_users = []
                    mention_counter.update(user['username'] for user in quoted_mentioned_users)

            # Obtain top 10 of more mentioned users
            return mention_counter.most_common(10)

In [ ]:
q3_time(file_path)

### As well as in the other exercises, for the memory implementation I used generators to maintain the memory with an efficient use.

In [ ]:
def mention_generator(file_path):
    """Generator to process mentions of each tweet and their quotedTweet."""
    with open(file_path, 'r') as f:
            # Read the file line by line
            for line in f:
                # Fix and convert each line in a JSON object
                line = line.strip()
                if not line:
                    # Ignore empty lines
                    continue

                # Validate the object to process correctly
                if line.startswith('{') and line.endswith('}'):
                    try:
                        tweet = json.loads(line)
                    except json.JSONDecodeError as e:
                        print(f'Error decoding the line: {line.strip()} - {e}')

                # Process mentions in 'mentionedUsers' from main tweet
                mentioned_users = tweet.get('mentionedUsers', [])
                if mentioned_users is None:
                    mentioned_users = []
                for user in mentioned_users:
                    yield user['username']

                # Process mentions in 'mentionedUsers' from 'quotedTweet' if it exists
                quoted_tweet = tweet.get('quotedTweet')
                if quoted_tweet:
                    quoted_mentioned_users = quoted_tweet.get('mentionedUsers', [])
                    if quoted_mentioned_users is None:
                        quoted_mentioned_users = []
                    for user in quoted_mentioned_users:
                        yield user['username']

def q3_memory(file_path):
    mention_counter = Counter()

    # Use generator to process mentions
    for username in mention_generator(file_path):
        mention_counter[username] += 1

    # Obtain top 10 of more mentioned users
    return mention_counter.most_common(10)

In [ ]:
q3_memory(file_path)

In [ ]:
%snakeviz q3_time(file_path)

In [ ]:
%snakeviz q3_memory(file_path)

In [ ]:
%%memray_flamegraph
def q3_time(file_path):
    mention_counter = Counter()

    with open(file_path, 'r') as f:
            # Read the file line by line
            for line in f:
                # Fix and convert each line in a JSON object
                line = line.strip()
                if not line:
                    # Ignore empty lines
                    continue

                # Validate the object to process correctly
                if line.startswith('{') and line.endswith('}'):
                    try:
                        tweet = orjson.loads(line)
                    except json.JSONDecodeError as e:
                        print(f'Error decoding the line: {line.strip()} - {e}')

                # Process mentions in 'mentionedUsers' from main tweet
                mentioned_users = tweet.get('mentionedUsers', [])
                if mentioned_users is None:
                    mentioned_users = []
                mention_counter.update(user['username'] for user in mentioned_users)

                # Process mentions in 'mentionedUsers' from 'quotedTweet' if it exists
                quoted_tweet = tweet.get('quotedTweet')
                if quoted_tweet:
                    quoted_mentioned_users = quoted_tweet.get('mentionedUsers', [])
                    if quoted_mentioned_users is None:
                        quoted_mentioned_users = []
                    mention_counter.update(user['username'] for user in quoted_mentioned_users)

            # Obtain top 10 of more mentioned users
            return mention_counter.most_common(10)

q3_time(file_path)

In [ ]:
%%memray_flamegraph
def mention_generator(file_path):
    """Generator to process mentions of each tweet and their quotedTweet."""
    with open(file_path, 'r') as f:
            # Read the file line by line
            for line in f:
                # Fix and convert each line in a JSON object
                line = line.strip()
                if not line:
                    # Ignore empty lines
                    continue

                # Validate the object to process correctly
                if line.startswith('{') and line.endswith('}'):
                    try:
                        tweet = json.loads(line)
                    except json.JSONDecodeError as e:
                        print(f'Error decoding the line: {line.strip()} - {e}')

                # Process mentions in 'mentionedUsers' from main tweet
                mentioned_users = tweet.get('mentionedUsers', [])
                if mentioned_users is None:
                    mentioned_users = []
                for user in mentioned_users:
                    yield user['username']

                # Process mentions in 'mentionedUsers' from 'quotedTweet' if it exists
                quoted_tweet = tweet.get('quotedTweet')
                if quoted_tweet:
                    quoted_mentioned_users = quoted_tweet.get('mentionedUsers', [])
                    if quoted_mentioned_users is None:
                        quoted_mentioned_users = []
                    for user in quoted_mentioned_users:
                        yield user['username']

def q3_memory(file_path):
    mention_counter = Counter()

    # Use generator to process mentions
    for username in mention_generator(file_path):
        mention_counter[username] += 1

    # Obtain top 10 of more mentioned users
    return mention_counter.most_common(10)

q3_memory(file_path)

### For the third exercise, we see that in terms of time, the execution time for the q3_time function is lower. In terms of memory, we see that both use approximately 883 KiB of memory; this, like in the previous case, may be due to the fact that both implementations process the file line by line and store it in structures that are already optimized for memory usage. Since neither of the functions loads the file completely, the usage is very similar in that they access the necessary elements in each iteration.

## Test functions from files

In [ ]:
from q1_time import q1_time
from q1_memory import q1_memory
from q2_time import q2_time
from q2_memory import q2_memory
from q3_time import q3_time
from q3_memory import q3_memory

In [ ]:
file_path = "farmers-protest-tweets-2021-2-4.json"

In [ ]:
q1_time(file_path=file_path)

In [ ]:
q1_memory(file_path=file_path)

In [ ]:
q2_time(file_path=file_path)

In [ ]:
q2_memory(file_path=file_path)

In [ ]:
q3_time(file_path=file_path)

In [ ]:
q3_memory(file_path=file_path)